In [ ]:
!git clone https://github.com/Cld338/Aspect-Term-Extraction-and-Analysis-Custom ATE
!pip install transformers
!pip install sentencepiece

In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1', last_hidden_states=True)
model = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)

In [ ]:
inputs = tokenizer("목차를 봤을 때 기억에 오래 남아요", return_tensors="pt")

{'input_ids': tensor([[   2, 2068, 7389, 6116, 2421, 7088, 1844, 1291, 6896, 3422, 1420, 6999,
            3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Bert(nn.Module):
    def __init__(
            self,
            model_name,
            n_classes,
            hidden_size,
            n_layers,
            dropout_p,
    ):
        self.model_name = model_name
        self.n_class = n_classes
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p

        super().__init__()

        self.bertmodel = BertModel.from_pretrained(self.model_name,  output_attentions = True, output_hidden_states=True)

        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout_p),
            nn.Linear(self.bertmodel.config.hidden_size, self.n_class)
        )

    def forward(self, x):
       
        # |x| = (batch_size, length)
        # encoder_layer= self.bertmodel(x, attention_mask)
        pooled_output = self.bertmodel(x)[1]
      
        x = self.classifier(pooled_output)

        return x

In [ ]:
model = bert(
            'skt/kobert-base-v1',
            3,
            hidden_size = 100,
            n_layers = 3,
            dropout_p =.25,
        )
encoding = tokenizer("나는 할 수 있다", return_tensors="pt")

x= encoding['input_ids']

outputs = model(x)
print(outputs)